In [44]:
class Person(object):
    def __init__(self, name, age, gender):
        self.name = name
        self.age = age
        self.gender = gender

In [45]:
import copy
import csv
import os
import zipfile
class Josephus:
    '约瑟夫环'

    def __init__(self):
        self.__people = []
        self.step = 0
        self.start = 1

    def append(self, obj):
        self.__people.append(obj)

    def pop(self):
        self.__people.pop(0)

    def query_list(self):
        return self.__people

    def len_list(self):
        return len(self.__people)

    def next_bymod(self):
        temp = copy.copy(self.__people)
        length = len(temp)
        if length == 0:
            return None
        id_ = self.start

        for i in range(length):
            id_ = (id_ + self.step - 1) % len(temp)
            index = temp.pop(id_)
            yield index

    def next_circular(self):
        temp = copy.copy(self.__people)
        if len(temp) == 0:
            return None

        index = 1
        start = self.start % len(temp)
        temp = temp[start:] + temp[:start]

        while index:
            head = (self.step-1) % len(temp)
            temp = temp[head:] + temp[:head]
            index = temp.pop(0)
            yield index

    @classmethod
    def create_from_file(cls, path):
        file_type = os.path.splitext(path)[1]
        if file_type == '.txt': 
            return cls.__create_from_txt(path)
        if file_type == '.csv':
            return cls.__create_from_csv(path)
        if file_type == '.zip':
            return cls.__create_from_zip(path)

    @classmethod
    def __create_from_txt(cls, path):
        obj = cls()
        
        with open(path, 'r', encoding='utf-8') as fp:
            read_txt = fp.readlines()
            for row in read_txt:
                index = row.strip().split(',')
                obj.append(Person(index[0], index[1], index[2]))

        return obj

    @classmethod
    def __create_from_csv(cls, path):
        obj = cls()

        with open(path, 'r') as fp:
            read_csv = csv.reader(fp)
            for index in read_csv:#type(index) = 'list'
                obj.append(Person(index[0], index[1], index[2]))

        return obj

    @classmethod
    def __create_from_zip(cls, path):
        with zipfile.ZipFile(path, 'r') as fp:
            file_list = fp.namelist()
            option = input("请从中选择一个文件{}". format(file_list))
            option_path = fp.extract(option)

        return cls.create_from_file(option_path)


In [46]:
path = 'people.zip'
ring = Josephus.create_from_file(path)
ring.start = 0
ring.step = 2


In [47]:
length = ring.len_list()

generator_bymod = ring.next_bymod()
for i in range(length):
    index = next(generator_bymod)
    print("第{}个出列的人:{}；年龄:{}；性别:{}". format(i+1, index.name, index.age, index.gender))

print('-'*30)

generator_circular = ring.next_circular()
for i in range(length):
    index = next(generator_circular)
    print("第{}个出列的人:{}；年龄:{}；性别:{}". format(i+1, index.name, index.age, index.gender))

第1个出列的人:Rick；年龄: 53；性别: 男
第2个出列的人:Beth；年龄: 34；性别: 女
第3个出列的人:Morty；年龄: 12；性别: 男
第4个出列的人:Jerry；年龄: 36；性别: 男
第5个出列的人:Summer；年龄: 16；性别: 女
------------------------------
第1个出列的人:Rick；年龄: 53；性别: 男
第2个出列的人:Beth；年龄: 34；性别: 女
第3个出列的人:Morty；年龄: 12；性别: 男
第4个出列的人:Jerry；年龄: 36；性别: 男
第5个出列的人:Summer；年龄: 16；性别: 女
